In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm

In [224]:
#loading the data and loosing all Null values
df = pd.read_csv(r'framingham_heart_disease.csv')
df = df[['male', 'heartRate', 'sysBP', 'totChol', 'BMI']]
original_size = df.shape[0]
df.dropna(how='any', inplace=True)
no_null_size = df.shape[0]
df.insert(loc=0, column='intercept', value=1)
print(f'Data size after dropping Null values is'
      f' {no_null_size / original_size * 100:0.3}% of the original data size')

Data size after dropping Null values is 98.4% of the original data size


In [225]:
def generate_sample(df, size):
    np.random.seed(4861351)
    flag = True
    sample = None
    while flag:  #will take another sample if there is one sex only
        sample = df.sample(size)
        female_count, male_count = sample[['male', 'BMI']].groupby('male').count()['BMI']
        if female_count != 0 and male_count != 0:
            if abs(female_count - male_count) <= size * 0.1:
                flag = False
            else:
                flag = True
    return sample

In [226]:
sample = generate_sample(df, 1000)
X_variables = ['intercept', 'heartRate', 'sysBP', 'totChol']
y_variable = 'BMI'
X = df[X_variables]
y = df[y_variable]
X_sample = sample[X_variables]
y_sample = sample[y_variable]


In [227]:
z_alpha = stats.norm.ppf(0.975)


#Q1.a
def calculate_beta(X, y):
    # Calculate MLE
    C = X.T @ X
    C_inv = np.linalg.inv(C)
    C_inv_X = C_inv @ X.T
    beta = C_inv_X @ y
    return np.round(beta, 4).to_numpy()


def beta_CI(X, y, beta_sample, x_variables):
    n = X.shape[0]
    p = X.shape[1]
    C = np.linalg.inv(X.T @ X)
    e = y - np.dot(X, beta_sample)  #residuals
    res_var_estimate = (1 / (n - p)) * (e.T @ e)  #sigma-hat squared
    CI = []
    for i, variable in enumerate(x_variables):
        std_estimate = np.sqrt(res_var_estimate * (C[i][i]))  #SE of Beta_i
        CI.append(np.round([beta_sample[i] - z_alpha * std_estimate,
                            beta_sample[i] + z_alpha * std_estimate], 4))
    return CI

In [228]:
#Q1.a
beta_sample = calculate_beta(X_sample, y_sample)
regular_CI = beta_CI(X_sample, y_sample, beta_sample, X_variables)
print(f'Beta = {beta_sample}')
for i in range(beta_sample.size):
    print(f"CI for beta_{i} is: {regular_CI[i]}")

Beta = [1.69997e+01 4.60000e-03 5.78000e-02 3.10000e-03]
CI for beta_0 is: [14.7841 19.2153]
CI for beta_1 is: [-0.0165  0.0257]
CI for beta_2 is: [0.0461 0.0695]
CI for beta_3 is: [-0.0025  0.0087]


In [229]:
argsort_y = np.argsort(sample['BMI'])
BMI_missing = []

for record, idx in zip(sample['BMI'], argsort_y):
    p = 0.2 + (0.0006 * idx)
    missing = np.random.binomial(1, p)
    if missing == 0:
        BMI_missing.append(record)
    else:
        BMI_missing.append(None)

sample['BMI_missing'] = BMI_missing

In [235]:
X_missing = X_sample[sample['BMI_missing'].isnull()==False]
y_missing = y_sample[sample['BMI_missing'].isnull()==False]
print(X_missing.shape)
beta_missing = calculate_beta(X_missing, y_missing)
CI_missing = beta_CI(X_missing, y_missing, beta_missing, X_variables)
print(f'Beta = {beta_missing}')
for i in range(beta_missing.size):
    print(f"CI for beta_{i} is: {CI_missing[i]}")

(508, 4)
Beta = [1.5287e+01 4.8000e-03 5.9100e-02 9.6000e-03]
CI for beta_0 is: [12.0055 18.5685]
CI for beta_1 is: [-0.0245  0.0341]
CI for beta_2 is: [0.0422 0.076 ]
CI for beta_3 is: [0.0011 0.0181]


$\arg \min_{\beta}{\|X\beta - \frac{RY}{\pi(W)}\|}^2$